<a href="https://colab.research.google.com/github/denisabrantesredis/denisd-GenAI-Workshop/blob/main/Labs/03-RAG_Images/03_Redis_RAG_Images.ipynb" target="_newt">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<div style="display:flex;width=100%;">
<img src="https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120" alt="Redis" width="90"/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://www.gstatic.com/devrel-devsite/prod/v0e0f589edd85502a40d78d7d0825db8ea5ef3b99ab4070381ee86977c9168730/cloud/images/cloud-logo.svg" alt="Google Cloud" width="140"/>
</div>

# Vector Similarity Search with Redis & Google Cloud - RAG for Images

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/redis_gcp.png?raw=true" alt="Redis and Google Cloud" align="center"/>

[Try a similar app with an always-on demo](https://ecommerce.redisventures.com/)

In this notebook, we will build a RAG use case using data from a web page. Redis will be used as the Vector Database and Cache for our use case, while Google Gemini is the LLM that will help generate the answers to the user's questions.

The dataset for this lab contains images of products like shoes, watches, clothes, etc. We will use Google Gemini to provide a description of each product that we can use as metadata.

## Installing the Pre-Reqs

In [ ]:
!pip install -q sentence-transformers==3.0.1 >> /.tmp
!pip install -q redis==5.0.8 >> /.tmp
!pip install -q redisvl==0.3.5 >> /.tmp
!pip install -q langchain==0.2.16 >> /.tmp
!pip install -q langchain-core==0.3.6 >> /.tmp
!pip install -q langchain-huggingface==0.0.3 >> /.tmp
!pip install -q langchain-redis==0.0.4 >> /.tmp
!pip install -q langchain-google-genai==2.0.0 >> /.tmp
!pip install -q langchain_experimental==0.3.2 >> /.tmp
!pip install -q open-clip-torch==2.26.1 >> /.tmp
!pip install -q git+https://github.com/openai/CLIP.git >> /.tmp

In [ ]:
# patch an issue with RedisVL
!wget https://github.com/denisabrantesredis/denisd-GenAI-Workshop/raw/refs/heads/main/_assets/files/semantic.py
!rm /usr/local/lib/python3.10/dist-packages/redisvl/extensions/llmcache/semantic.py
!cp semantic.py /usr/local/lib/python3.10/dist-packages/redisvl/extensions/llmcache/

### Installing Redis Stack Locally
If you are not using Redis Cloud as a database, uncomment and run the code below to install Redis locally. Then set your connection to 127.0.0.1

In [ ]:
# %%sh
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg 
# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list 
# sudo apt-get update  > /dev/null 2>&1
# sudo apt-get install redis-stack-server  > /dev/null 2>&1
# redis-stack-server --daemonize yes 

### Loading Required Packages

In [ ]:
import os
import glob
import json
import redis
import torch
import base64
import random
from typing import Any, Dict
from IPython.display import Image
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, model_serializer
from langchain.output_parsers import PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

## Part 1 - Prepare the Environment

### Step 1: Download Dataset from Github

For performance reasons, we will only be working with 20 images. In this github repo, you can find a zip file with 100 images that can be used instead, if you want to test this code in an environment with more resources, like Google Vertex AI.

We will download the images that will be stored in Redis as vectors, as well as a smaller dataset of images that will be used for semantic search.

In [ ]:
if not os.path.exists("./img_20"):
  !wget https://github.com/denisabrantesredis/denisd-GenAI-Workshop/raw/refs/heads/main/_assets/files/img20.zip
  !unzip img20.zip

In [ ]:
if not os.path.exists("./img_search_20"):
  !wget https://github.com/denisabrantesredis/denisd-GenAI-Workshop/raw/refs/heads/main/_assets/files/img_search20.zip
  !unzip img_search20.zip

### Step 2: Setting up the Redis connection and GCP API Key

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_secrets.png?raw=true" alt="Callout - Use Google Colab secrets instead"/>

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    if userdata.get('GOOGLE_API_KEY'):
      os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    else:
      os.environ["GOOGLE_API_KEY"] = "<insert API key here>"

if userdata.get('REDIS_HOST'):
  REDIS_HOST = userdata.get('REDIS_HOST')
else:
  REDIS_HOST="127.0.0.1"

if userdata.get('REDIS_PORT'):
  REDIS_PORT = userdata.get('REDIS_PORT')
else:
  REDIS_PORT=12000

if userdata.get('REDIS_PASSWORD'):
  REDIS_PASSWORD = userdata.get('REDIS_PASSWORD')
else:
  REDIS_PASSWORD="password"

REDIS_URL = f"redis://default:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

#### Testing the Connection to Redis

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_connection.png?raw=true" alt="Callout - Make sure connection works"/>

In [ ]:
r = redis.from_url(REDIS_URL)

if r.ping():
    print("Connection successful!")
else:
    print("Connection issue!")

### Step 3: Load the list of images

This lab can greatly benefit from running on a T4 GPU. However, seeing as GPUs are not guaranteed in the free tier, the lab was designed to also run on CPUs, albeit slower.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
filenames = glob.glob("./img_20/*")
len(filenames)

#### Step 4: Load Gemini and get it to describe an image

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.5,
    top_p=0.95,
    top_k=64,
    max_output_tokens=8192
    )

Display the image that will be sent to the model

In [ ]:
with open(filenames[0], "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode()

Image(filenames[0])

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_geminiimage.png?raw=true" alt="Callout - Upload Image to Gemini"/>

In [ ]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the object in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)

Call the model and print the response

In [ ]:
# Invoke the model with the message
response = llm.invoke([message])

# Print the model's response
print(response.content)

&nbsp;

## Part 2: Categorizing the Images

Usually, a dataset of images would have a curated set of metadata attributes, that would be used as metadata for hybrid searches. In this lab, however, we will generate metadata for each image using Google Gemini to provide a description of the image and its key characteristics.

### Prepare a list of images on base64 format

In [ ]:
image_list = []
for i in range(len(filenames)):
    this_image_path = filenames[i]
    with open(this_image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")
        image_list.append(image_data)

### Visualizing the search results with the score for each result

In [ ]:
print(f"Search results for '{query}':")
for doc in results:
    print("----")
    print(f"Score: {doc[1]} - {doc[0].page_content} (Source: {doc[0].metadata['url']})")

&nbsp;

## Part 5: Using a LLM

In this lab, we will use the Gemini Pro 1.5 model from Google to generate a response to the user, based on the documents retrieved from Redis. The GCP API Key that we set before is required to allow access to the model.

### Step 1: Load the Model

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.5,
    top_p=0.95,
    top_k=64,
    max_output_tokens=8192
    )

### Step 2: Prepare a list with the text from the documents retrieved by the vector search

We will ask the model to respond to the user's questions. To help with the answer, we want to provide the text from the documents that were retrieved by the semantic search.

In [ ]:
text_list = []
distance_list = []

for node in results:
    text_list.append(node[0].page_content)
    distance = node[1]
    distance_list.append(distance)

Print the list that will be sent to the model.

In [ ]:
text_list

### Step 3: Prepare the Prompt

Since this is just a lab, we will keep the prompt very simple, with just basic instructions for the model to answer based on the documents from the semantic search, and to stick to the documents for the response. Production prompts will benefit from more sophisticated prompts, as well as other controls like guardrails, etc.

In [ ]:
def get_system_template(text_list, query):
  system_template = """
  Your task is to answer questions by using a given context.

  Don't invent anything that is outside of the context.

  %CONTEXT%
  {context}

  """
  messages = [
      SystemMessage(content=system_template.format(context=text_list)),
      HumanMessage(content=query)
  ]

  return messages

In [ ]:
messages = get_system_template(text_list, query)

### Step 4: Invoke the Model

Since we are not using Redis as cache, the model will be called every time, even if the same question (or a similar) is asked multiple times.

In [ ]:
timer_start = time.perf_counter()
llm_response = llm.invoke(messages)
timer_end = time.perf_counter()
total_time = round(timer_end - timer_start, 4)
print(f"Total Time: {total_time}s")

Visualizing the model response:

In [ ]:
llm_response.content

&nbsp;

## Part 6: Leveraging Redis for Basic Cache

Redis can be used not only as the Vector Database, but also as a cache to store responses from the Large Language Model, which can significantly improve user experience, by retrieving responses in milliseconds instead of seconds.

In [ ]:
from langchain_redis import RedisCache
from langchain.globals import set_llm_cache

To use Redis as a cache, we only need 2 lines of code:

In [ ]:
redis_cache = RedisCache(redis_url=REDIS_URL)
set_llm_cache(redis_cache)

We will repeat the same question from before. Since we have just enabled the cache, it will be empty, which means that this next question will require a vector search and will need to go through the Large Language Model again.

In [ ]:
query = "How does Redis Insight make RDI simpler?"

In [ ]:
timer_start = time.perf_counter()
result_nodes = vector_store.similarity_search_with_score(query)
timer_end = time.perf_counter()
total_time = round(timer_end - timer_start, 4)
print(f"Total Time: {total_time}s")

Prepare the list of texts to send to the LLM:

In [ ]:
text_list = []
distance_list = []

for node in result_nodes:
    text_list.append(node[0].page_content)
    distance = node[1]
    distance_list.append(distance)

Display the search results:

In [ ]:
print(f"--> Total Documents Found: {len(result_nodes)}")
for node in result_nodes:
  print(f"--> {node[1]} | {node[0].page_content}")

Prepare the prompt:

In [ ]:
messages = get_system_template(text_list, query)

Call the model (the `invoke` function will check and populate the cache automatically):

In [ ]:
timer_start = time.perf_counter()
llm_response = llm.invoke(messages)
timer_end = time.perf_counter()
total_time = round(timer_end - timer_start, 4)
print(f"Total Time: {total_time}s")

Print the LLM response:

In [ ]:
llm_response.content

&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

A new document should appear on Redis, of type JSON. This is the cached response from the LLM.
Notice that the key is made from a long this; this is a hash of the question.

Because this is a basic cache, questions from the user will be hashed and compared against the key, which means that for this basic cache, questions must match exactly in order to be used.

&nbsp;


#### Repeating the question to fetch results from the cache

When we ask exactly the same question as before, it should trigger a cache hit, meaning we will receive the answer from the Redis cache, much faster than calling the model.

In [ ]:
timer_start = time.perf_counter()
llm_response = llm.invoke(messages)
timer_end = time.perf_counter()
total_time = round(timer_end - timer_start, 4)
print(f"Total Time: {total_time}s")

Print the cached response:

In [ ]:
llm_response.content

&nbsp;

#### Asking the same question (worded differently) will cause a cache miss

If the question is not an exact match, it will cause a cache miss. This might be an issue with most of the RAG use cases, which is why we will be exploring Semantic Cache next.

In [ ]:
# original query = "How does Redis Insight make RDI simpler?"
query = "What does Redis Insight do to make RDI simpler?"

Prepare the prompt with the new query (PS: we're skipping the vector search on purpose)

In [ ]:
messages = get_system_template(text_list, query)

Call the model:

In [ ]:
timer_start = time.perf_counter()
llm_response = llm.invoke(messages)
timer_end = time.perf_counter()
total_time = round(timer_end - timer_start, 4)
print(f"Total Time: {total_time}s")

Print the response:

In [ ]:
llm_response.content

&nbsp;

## Part 7 - Leveraging Redis for Semantic Cache

The Semantic Cache will generate vectors for each prompt, and store the response from the LLM. That way, new prompts are converted into vectors automatically and a semantic search is executed on Redis, looking for similar questions.

It is possible to set the threshold for for semantic search; for this lab, we are using 20%. In your project, you can run multiple tests with different thresholds, to determined what works best for your use case.

In [ ]:
from langchain_redis import RedisSemanticCache

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_threshold.png?raw=true" alt="Callout - Semantic Threshold"/>

In [ ]:
redis_cache = RedisSemanticCache(redis_url=REDIS_URL, embeddings=embeddings, distance_threshold=0.2)
set_llm_cache(redis_cache)

Since the Semantic Cache is new, it will be empty. We will ask the original question first, to generate the cache entry:

In [ ]:
query = "How does Redis Insight make RDI simpler?"

Prepare the prompt:

In [ ]:
messages = get_system_template(text_list, query)

Invoke the model (it will cause a cache miss):

In [ ]:
timer_start = time.perf_counter()
llm_response = llm.invoke(messages)
timer_end = time.perf_counter()
total_time = round(timer_end - timer_start, 4)
print(f"Total Time: {total_time}s")

Print the response:

In [ ]:
llm_response.content

&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

A new Hash document will appear in Redis, with a key prefix of `llmcache`. This is the cached prompt, which includes the question and the answer. The `invoke` function will run a semantic search for these documents, to look for similar questions.

&nbsp;

#### Ask a similar question to trigger a cache hit

In [ ]:
query = "What does Redis Insight do to make RDI simpler?"

Prepare the prompt:

In [ ]:
messages = get_system_template(text_list, query)

Invoke the model:

In [ ]:
timer_start = time.perf_counter()
llm_response = llm.invoke(messages)
timer_end = time.perf_counter()
total_time = round(timer_end - timer_start, 4)
print(f"Total Time: {total_time}s")

Print the response:

In [ ]:
llm_response.content

&nbsp;


&nbsp;



# Congrats, this is the end of the lab!!